In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import string
import pandas as pd
import cv2

def mymodel():
    model = Sequential()

    model.add(Conv2D(16,(3,3),activation='relu',input_shape=(128,128,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Conv2D(64,(5,5),activation='relu',padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(3))

    model.add(Flatten())

    model.add(Dense(256,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(1,activation='sigmoid'))
    return model
    
#1. 데이터
x_train = np.load('/content/drive/MyDrive/dacon/npy/x_train_128.npy')
x_test = np.load('/content/drive/MyDrive/dacon/npy/x_test_128.npy')
answer = pd.read_csv('/content/drive/MyDrive/dacon/dirty_mnist_answer.csv')
sub = pd.read_csv('/content/drive/MyDrive/dacon/sample_submission.csv', header = 0)

# print(x_train.shape) (50000, 128, 128, 1)
# print(np.max(x_train[0])) # 1.0

alphabets = string.ascii_lowercase
alphabets = list(alphabets)

val = []
acc = []
for alphabet in alphabets:
    y_train = answer.loc[:,alphabet].to_numpy()

    #2. 모델
    model = mymodel()

    #3. 컴파일 훈련
    es = EarlyStopping(patience = 20)
    lr = ReduceLROnPlateau(patience = 7, factor = 0.25, verbose = 1)
    cp = ModelCheckpoint(f'/content/drive/MyDrive/dacon/mcp/{alphabet}.hdf5')
    model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['acc'])
    history = model.fit(x_train, y_train, epochs = 10000, batch_size = 128, validation_split = 0.2, callbacks = [es,cp,lr])
    val.append(history.history['val_loss'][-21])
    acc.append(history.history['val_acc'][-21])

print('loss : ', val)
print('acc : ', acc)